In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score
import joblib

In [2]:
# Set random seed for reproducibility
np.random.seed(42)
n_samples = 5000

print("🔧 Generating emergency landing risk training data...")

# Generate base features
data = {
    "flight_id": [f"FL{str(i).zfill(4)}" for i in range(n_samples)],
    "engine_health": np.round(np.random.uniform(50, 100, n_samples), 1),
    "vibration_level": np.round(np.random.uniform(0, 10, n_samples), 1),
    "fuel_pressure": np.round(np.random.uniform(60, 100, n_samples), 1),
    "hydraulic_pressure": np.round(np.random.uniform(70, 100, n_samples), 1),
    "oil_temperature": np.round(np.random.uniform(60, 120, n_samples), 1),
    "cabin_pressure": np.round(np.random.uniform(80, 100, n_samples), 1),
    "fuel_quantity": np.round(np.random.uniform(10, 100, n_samples), 1),
    "electrical_system_health": np.round(np.random.uniform(70, 100, n_samples), 1),
    "flight_control_response": np.round(np.random.uniform(70, 100, n_samples), 1),
    "weather_severity": np.random.randint(0, 11, n_samples),
    "altitude": np.random.randint(5000, 40000, n_samples),
    "airspeed": np.random.randint(200, 600, n_samples),
    "turbulence_level": np.random.randint(0, 11, n_samples),
    "visibility": np.random.randint(1, 11, n_samples),
    "wind_shear": np.random.randint(0, 11, n_samples),
    "time_of_day": np.random.choice(["Morning", "Afternoon", "Evening", "Night"], n_samples),
    "phase_of_flight": np.random.choice(["Takeoff", "Climb", "Cruise", "Descent", "Approach", "Landing"], n_samples),
    "pilot_experience_years": np.random.randint(2, 30, n_samples),
    "aircraft_age_years": np.round(np.random.uniform(1, 25, n_samples), 1)
}

df = pd.DataFrame(data)
print(f"✅ Generated {len(df)} samples")
df.head()


🔧 Generating emergency landing risk training data...
✅ Generated 5000 samples


,flight_id,engine_health,vibration_level,fuel_pressure,hydraulic_pressure,oil_temperature,cabin_pressure,fuel_quantity,electrical_system_health,flight_control_response,weather_severity,altitude,airspeed,turbulence_level,visibility,wind_shear,time_of_day,phase_of_flight,pilot_experience_years,aircraft_age_years
0,FL0000,68.7,3.9,74.9,85.0,103.8,81.8,67.4,88.3,79.0,9,34495,259,9,3,4,Afternoon,Descent,22,18.9
1,FL0001,97.5,4.7,73.3,92.4,71.1,81.2,51.3,71.2,72.8,9,37709,479,4,10,8,Evening,Climb,24,2.0
2,FL0002,86.6,8.5,67.0,86.9,80.8,92.1,96.8,88.4,73.8,6,30103,216,8,10,0,Evening,Approach,5,2.9
3,FL0003,79.9,3.4,84.3,72.5,99.8,99.3,29.7,72.7,75.4,3,12173,520,3,7,8,Night,Approach,16,18.1
4,FL0004,57.8,8.7,79.1,75.6,88.9,90.1,62.9,91.3,76.1,8,21417,366,7,10,3,Evening,Landing,28,16.5


In [3]:

engine_norm = 1 - (df["engine_health"] / 100)  # Inverted (lower health = higher risk)
vibration_norm = df["vibration_level"] / 10
fuel_pressure_norm = 1 - (df["fuel_pressure"] / 100)  # Inverted
hydraulic_norm = 1 - (df["hydraulic_pressure"] / 100)  # Inverted
oil_temp_norm = np.abs(df["oil_temperature"] - 90) / 60  # Optimal at 90°C
cabin_pressure_norm = 1 - (df["cabin_pressure"] / 100)  # Inverted
fuel_quantity_norm = 1 - (df["fuel_quantity"] / 100)  # Inverted (low fuel = high risk)
electrical_norm = 1 - (df["electrical_system_health"] / 100)  # Inverted
flight_control_norm = 1 - (df["flight_control_response"] / 100)  # Inverted
weather_norm = df["weather_severity"] / 10
turbulence_norm = df["turbulence_level"] / 10
visibility_norm = 1 - (df["visibility"] / 10)  # Inverted
wind_shear_norm = df["wind_shear"] / 10

# Phase of flight risk (some phases are more critical)
phase_risk = df["phase_of_flight"].map({
    "Cruise": 0.0,
    "Climb": 0.1,
    "Descent": 0.1,
    "Takeoff": 0.2,
    "Approach": 0.2,
    "Landing": 0.25
})

# Time of day risk (night = slightly higher risk)
time_risk = df["time_of_day"].map({
    "Morning": 0.0,
    "Afternoon": 0.0,
    "Evening": 0.05,
    "Night": 0.1
})

# Pilot experience (less experience = higher risk)
pilot_risk = np.clip(1 - (df["pilot_experience_years"] / 30), 0, 1) * 0.1

# Aircraft age risk
age_risk = (df["aircraft_age_years"] / 25) * 0.05

# Altitude risk (too low or too high is risky)
altitude_risk = np.where(df["altitude"] < 10000, 0.1,
                np.where(df["altitude"] > 35000, 0.05, 0.0))

# Calculate comprehensive risk score with realistic weights
risk_score = (
    0.25 * engine_norm +                    # 25% weight on engine health (CRITICAL)
    0.15 * vibration_norm +                 # 15% weight on vibration
    0.10 * fuel_pressure_norm +             # 10% weight on fuel pressure
    0.08 * hydraulic_norm +                 # 8% weight on hydraulics
    0.05 * oil_temp_norm +                  # 5% weight on oil temperature
    0.05 * cabin_pressure_norm +            # 5% weight on cabin pressure
    0.08 * fuel_quantity_norm +             # 8% weight on fuel quantity
    0.06 * electrical_norm +                # 6% weight on electrical
    0.10 * flight_control_norm +            # 10% weight on flight controls (CRITICAL)
    0.08 * weather_norm +                   # 8% weight on weather
    0.05 * turbulence_norm +                # 5% weight on turbulence
    0.03 * visibility_norm +                # 3% weight on visibility
    0.04 * wind_shear_norm +                # 4% weight on wind shear
    phase_risk +                            # Phase adjustment
    time_risk +                             # Time of day adjustment
    pilot_risk +                            # Pilot experience adjustment
    age_risk +                              # Aircraft age adjustment
    altitude_risk                           # Altitude adjustment
)

# Add realistic random variation
risk_score += np.random.normal(0, 0.06, n_samples)
risk_score = np.clip(risk_score, 0, 1)

# Convert to probability with sigmoid for smooth distribution
probability = 1 / (1 + np.exp(-12 * (risk_score - 0.5)))

# Create binary target (emergency landing risk)
threshold = 0.45
df["emergency_probability"] = probability
df["emergency_risk_binary"] = (probability > threshold).astype(int)

# Create risk levels
def assign_risk_level(prob):
    if prob < 0.25:
        return "Low"
    elif prob < 0.5:
        return "Moderate"
    elif prob < 0.75:
        return "High"
    else:
        return "Critical"

df["emergency_risk_level"] = df["emergency_probability"].apply(assign_risk_level)

# Create recommended actions
def assign_action(prob):
    if prob < 0.25:
        return "Continue Normal Operations"
    elif prob < 0.5:
        return "Increase Monitoring"
    elif prob < 0.75:
        return "Prepare Diversion Plan"
    else:
        return "Initiate Emergency Procedures"

df["recommended_action"] = df["emergency_probability"].apply(assign_action)

print(f"\n📊 Class Distribution:")
print(df["emergency_risk_binary"].value_counts())
print(f"\nPositive class rate: {df['emergency_risk_binary'].mean():.2%}")
print(f"\nSample predictions range: {probability.min():.2f} to {probability.max():.2f}")

print(f"\n📊 Risk Level Distribution:")
print(df["emergency_risk_level"].value_counts())

df[["flight_id", "emergency_probability", "emergency_risk_level", "recommended_action"]].head(10)



📊 Class Distribution:
1    4175
0     825
Name: emergency_risk_binary, dtype: int64

Positive class rate: 83.50%

Sample predictions range: 0.01 to 1.00

📊 Risk Level Distribution:
Critical    2947
High        1067
Moderate     646
Low          340
Name: emergency_risk_level, dtype: int64


,flight_id,emergency_probability,emergency_risk_level,recommended_action
0,FL0000,0.270929,Moderate,Increase Monitoring
1,FL0001,0.470072,Moderate,Increase Monitoring
2,FL0002,0.878527,Critical,Initiate Emergency Procedures
3,FL0003,0.970255,Critical,Initiate Emergency Procedures
4,FL0004,0.848713,Critical,Initiate Emergency Procedures
5,FL0005,0.862496,Critical,Initiate Emergency Procedures
6,FL0006,0.884957,Critical,Initiate Emergency Procedures
7,FL0007,0.741844,High,Prepare Diversion Plan
8,FL0008,0.482114,Moderate,Increase Monitoring
9,FL0009,0.766189,Critical,Initiate Emergency Procedures


In [4]:

# Prepare features and target
X = df.drop(columns=["flight_id", "emergency_probability", "emergency_risk_binary", 
                     "emergency_risk_level", "recommended_action"])
y_binary = df["emergency_risk_binary"]
y_risk_level = df["emergency_risk_level"]

# Define preprocessing
categorical_cols = ["time_of_day", "phase_of_flight"]
numerical_cols = list(X.columns.difference(categorical_cols))

print(f"Numerical features: {numerical_cols}")
print(f"Categorical features: {categorical_cols}")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
    ]
)

# Create binary classification model (for emergency probability)
binary_model = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42
)

# Create multi-class model (for risk levels)
risk_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    class_weight='balanced'
)

# Create pipelines
binary_pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", binary_model)
    ]
)

risk_pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", risk_model)
    ]
)

print("✅ Pipelines created successfully")

Numerical features: ['aircraft_age_years', 'airspeed', 'altitude', 'cabin_pressure', 'electrical_system_health', 'engine_health', 'flight_control_response', 'fuel_pressure', 'fuel_quantity', 'hydraulic_pressure', 'oil_temperature', 'pilot_experience_years', 'turbulence_level', 'vibration_level', 'visibility', 'weather_severity', 'wind_shear']
Categorical features: ['time_of_day', 'phase_of_flight']
✅ Pipelines created successfully


In [5]:
# Split data with stratification
X_train, X_test, y_train_binary, y_test_binary, y_train_risk, y_test_risk = train_test_split(
    X, y_binary, y_risk_level, test_size=0.2, random_state=42, stratify=y_binary
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

# Train binary model
print("\n🚀 Training Binary Classification Model (Emergency Probability)...")
binary_pipeline.fit(X_train, y_train_binary)
print("✅ Training complete!")

# Make predictions
y_prob_binary = binary_pipeline.predict_proba(X_test)[:, 1]
y_pred_binary = binary_pipeline.predict(X_test)

# Evaluate performance
print("\n" + "="*60)
print("📈 BINARY MODEL PERFORMANCE (Emergency Probability)")
print("="*60)
print(f"\nROC-AUC Score: {roc_auc_score(y_test_binary, y_prob_binary):.4f}")
print(f"Accuracy: {accuracy_score(y_test_binary, y_pred_binary):.4f}")

print("\n📊 Classification Report:")
print(classification_report(y_test_binary, y_pred_binary, 
                          target_names=['No Emergency Risk', 'Emergency Risk']))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test_binary, y_pred_binary))

# Check prediction distribution
print("\n📊 Prediction Probability Distribution on Test Set:")
prob_bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
prob_labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
prob_dist = pd.cut(y_prob_binary, bins=prob_bins, labels=prob_labels)
print(prob_dist.value_counts().sort_index())

Training set: 4000 samples
Test set: 1000 samples

🚀 Training Binary Classification Model (Emergency Probability)...
✅ Training complete!

📈 BINARY MODEL PERFORMANCE (Emergency Probability)

ROC-AUC Score: 0.9418
Accuracy: 0.9070

📊 Classification Report:
                   precision    recall  f1-score   support

No Emergency Risk       0.82      0.56      0.66       165
   Emergency Risk       0.92      0.98      0.95       835

         accuracy                           0.91      1000
        macro avg       0.87      0.77      0.81      1000
     weighted avg       0.90      0.91      0.90      1000


📊 Confusion Matrix:
[[ 92  73]
 [ 20 815]]

📊 Prediction Probability Distribution on Test Set:
0-20%       88
20-40%      17
40-60%      17
60-80%      43
80-100%    835
dtype: int64


In [6]:
print("\n🚀 Training Multi-Class Model (Risk Levels)...")
risk_pipeline.fit(X_train, y_train_risk)
print("✅ Training complete!")

# Make predictions
y_pred_risk = risk_pipeline.predict(X_test)
y_prob_risk = risk_pipeline.predict_proba(X_test)

# Evaluate performance
print("\n" + "="*60)
print("📈 RISK LEVEL MODEL PERFORMANCE")
print("="*60)
print(f"\nAccuracy: {accuracy_score(y_test_risk, y_pred_risk):.4f}")

print("\n📊 Classification Report:")
print(classification_report(y_test_risk, y_pred_risk))

print("\n📊 Confusion Matrix:")
cm = confusion_matrix(y_test_risk, y_pred_risk, labels=['Low', 'Moderate', 'High', 'Critical'])
cm_df = pd.DataFrame(cm, 
                     index=['Low', 'Moderate', 'High', 'Critical'],
                     columns=['Low', 'Moderate', 'High', 'Critical'])
print(cm_df)



🚀 Training Multi-Class Model (Risk Levels)...
✅ Training complete!

📈 RISK LEVEL MODEL PERFORMANCE

Accuracy: 0.6810

📊 Classification Report:
              precision    recall  f1-score   support

    Critical       0.83      0.87      0.85       594
        High       0.43      0.38      0.40       215
         Low       0.48      0.56      0.52        68
    Moderate       0.40      0.35      0.37       123

    accuracy                           0.68      1000
   macro avg       0.54      0.54      0.54      1000
weighted avg       0.67      0.68      0.67      1000


📊 Confusion Matrix:
          Low  Moderate  High  Critical
Low        38        22     7         1
Moderate   28        43    40        12
High       12        28    81        94
Critical    1        15    59       519


In [7]:
print("\n🧪 TESTING WITH SAMPLE SCENARIOS")
print("="*60)

test_cases = [
    {
        "name": "✅ Low Risk Flight (Normal Operations)",
        "data": {
            'engine_health': 95.0,
            'vibration_level': 1.5,
            'fuel_pressure': 98.0,
            'hydraulic_pressure': 97.0,
            'oil_temperature': 88.0,
            'cabin_pressure': 98.0,
            'fuel_quantity': 85.0,
            'electrical_system_health': 96.0,
            'flight_control_response': 98.0,
            'weather_severity': 2,
            'altitude': 35000,
            'airspeed': 480,
            'turbulence_level': 1,
            'visibility': 9,
            'wind_shear': 1,
            'time_of_day': 'Morning',
            'phase_of_flight': 'Cruise',
            'pilot_experience_years': 15,
            'aircraft_age_years': 5.0
        }
    },
    {
        "name": "⚠️ Moderate Risk Flight (Minor Issues)",
        "data": {
            'engine_health': 82.0,
            'vibration_level': 4.5,
            'fuel_pressure': 85.0,
            'hydraulic_pressure': 88.0,
            'oil_temperature': 102.0,
            'cabin_pressure': 92.0,
            'fuel_quantity': 45.0,
            'electrical_system_health': 86.0,
            'flight_control_response': 90.0,
            'weather_severity': 5,
            'altitude': 28000,
            'airspeed': 420,
            'turbulence_level': 4,
            'visibility': 6,
            'wind_shear': 4,
            'time_of_day': 'Evening',
            'phase_of_flight': 'Descent',
            'pilot_experience_years': 8,
            'aircraft_age_years': 12.0
        }
    },
    {
        "name": "🔴 High Risk Flight (Multiple Critical Issues)",
        "data": {
            'engine_health': 68.0,
            'vibration_level': 8.5,
            'fuel_pressure': 72.0,
            'hydraulic_pressure': 75.0,
            'oil_temperature': 115.0,
            'cabin_pressure': 82.0,
            'fuel_quantity': 18.0,
            'electrical_system_health': 74.0,
            'flight_control_response': 76.0,
            'weather_severity': 9,
            'altitude': 15000,
            'airspeed': 280,
            'turbulence_level': 8,
            'visibility': 2,
            'wind_shear': 8,
            'time_of_day': 'Night',
            'phase_of_flight': 'Approach',
            'pilot_experience_years': 4,
            'aircraft_age_years': 22.0
        }
    },
    {
        "name": "🚨 Critical Emergency (Severe Engine Failure)",
        "data": {
            'engine_health': 52.0,
            'vibration_level': 9.8,
            'fuel_pressure': 65.0,
            'hydraulic_pressure': 68.0,
            'oil_temperature': 118.0,
            'cabin_pressure': 78.0,
            'fuel_quantity': 12.0,
            'electrical_system_health': 70.0,
            'flight_control_response': 72.0,
            'weather_severity': 10,
            'altitude': 8000,
            'airspeed': 250,
            'turbulence_level': 9,
            'visibility': 1,
            'wind_shear': 9,
            'time_of_day': 'Night',
            'phase_of_flight': 'Landing',
            'pilot_experience_years': 3,
            'aircraft_age_years': 24.0
        }
    }
]

for test_case in test_cases:
    test_df = pd.DataFrame([test_case['data']])
    
    # Binary prediction
    prob_binary = binary_pipeline.predict_proba(test_df)[0][1] * 100
    pred_binary = "At Risk" if prob_binary > 50 else "Normal"
    
    # Risk level prediction
    risk_level = risk_pipeline.predict(test_df)[0]
    risk_probs = risk_pipeline.predict_proba(test_df)[0]
    risk_confidence = max(risk_probs) * 100
    
    # Recommended action
    if prob_binary < 25:
        action = "Continue Normal Operations"
    elif prob_binary < 50:
        action = "Increase Monitoring"
    elif prob_binary < 75:
        action = "Prepare Diversion Plan"
    else:
        action = "Initiate Emergency Procedures"
    
    print(f"\n{test_case['name']}")
    print(f"  Emergency Probability: {prob_binary:.1f}% ({pred_binary})")
    print(f"  Risk Level: {risk_level} (confidence: {risk_confidence:.1f}%)")
    print(f"  Recommended Action: {action}")



🧪 TESTING WITH SAMPLE SCENARIOS

✅ Low Risk Flight (Normal Operations)
  Emergency Probability: 0.0% (Normal)
  Risk Level: Low (confidence: 76.3%)
  Recommended Action: Continue Normal Operations

⚠️ Moderate Risk Flight (Minor Issues)
  Emergency Probability: 99.7% (At Risk)
  Risk Level: High (confidence: 45.5%)
  Recommended Action: Initiate Emergency Procedures

🔴 High Risk Flight (Multiple Critical Issues)
  Emergency Probability: 100.0% (At Risk)
  Risk Level: Critical (confidence: 94.3%)
  Recommended Action: Initiate Emergency Procedures

🚨 Critical Emergency (Severe Engine Failure)
  Emergency Probability: 100.0% (At Risk)
  Risk Level: Critical (confidence: 90.2%)
  Recommended Action: Initiate Emergency Procedures


In [8]:
# Save the trained models
binary_model_path = "emergency_landing_prob_model.pkl"
risk_model_path = "emergency_landing_risk_model.pkl"

joblib.dump(binary_pipeline, binary_model_path)
joblib.dump(risk_pipeline, risk_model_path)

# Save metadata
metadata = {
    "model_version": "1.0",
    "training_samples": len(X_train),
    "test_samples": len(X_test),
    "features": list(X.columns),
    "categorical_features": categorical_cols,
    "numerical_features": numerical_cols,
    "risk_levels": ["Low", "Moderate", "High", "Critical"],
    "actions": ["Continue Normal Operations", "Increase Monitoring", 
                "Prepare Diversion Plan", "Initiate Emergency Procedures"],
    "binary_model_accuracy": float(accuracy_score(y_test_binary, y_pred_binary)),
    "binary_model_auc": float(roc_auc_score(y_test_binary, y_prob_binary)),
    "risk_model_accuracy": float(accuracy_score(y_test_risk, y_pred_risk))
}

import json
with open("emergency_model_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("\n" + "="*60)
print("✅ MODELS SAVED SUCCESSFULLY!")
print("="*60)
print(f"\nFiles created:")
print(f"1. {binary_model_path}")
print(f"2. {risk_model_path}")
print(f"3. emergency_model_metadata.json")


✅ MODELS SAVED SUCCESSFULLY!

Files created:
1. emergency_landing_prob_model.pkl
2. emergency_landing_risk_model.pkl
3. emergency_model_metadata.json
